# Stage 1.1 : Data Process
---

---
## Contents
1. [Background](#Background)
    2. [Data Description](#Data_Description)
1. [Data transformation](#Data_transformation)
    1. Filter support data records for only 2015,2016 and 2017
    1. Filter based on relevant method of measure
    1. Combine Happiness datasets
    1. [Group by Average](#Group_by_Average ) 
    2. Outer Join support OECD datasets based on the `Location` and `year`
    3. Left Join (Return all rows from Main data) based on the `Location` and `year`
    4. [Match OECD and Happiness data by country code](#Using_country_code_to_match_datasets)
2. [Data Cleaning](#Data_Cleaning)
    1. Detect incorrect values
    2. Convert string type to numerical columns¶
    3. Check duplicate rows
    4. Check missing value
    5. Handle missing value 
       - Drop rows 
       - Fill missing value

3. [Data Analysis](#Data_Analysis)
    See Stage_1.2_Exploratory_Data_Analysis.ipynb




### Data Description
**Main datasets**: Happiness level rank in 2015, 2016, and 2017

**Support datasets**: 
- Body Health: alcohol_consumption, overweight_population
- Mental Health: suicide_statistics
- Economy: disposable_income
- Environment: air_pollution
- Education: adult_education
- Jobs: avg_annual_hours, employment_ratios
- Society: tourismGDP
- Country_code (Alpha-3)
---

In [1]:
import requests
import json
import pandas as pd
import re
import math
import warnings
warnings.filterwarnings('ignore')
import csv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from scipy.stats import norm, skew
from sklearn.linear_model import LogisticRegression
pd.set_option('display.max_columns', 50) # show all columns
pd.set_option('display.max_rows', 500) # show all rows
from matplotlib.colors import LogNorm

### Load data

In [2]:
## Load world hapiness data
happiness_2015 = pd.read_csv("../data/world-happiness/2015.csv")
happiness_2016 = pd.read_csv("../data/world-happiness/2016.csv")
happiness_2017 = pd.read_csv("../data/world-happiness/2017.csv")

In [3]:
## Rename 2017 dataset since it columns are inconsistent with 2015 and 2016 dataset.
happiness_2017 = happiness_2017.rename(columns = { 
                                  "Happiness.Rank":"Happiness Rank",
                                  'Happiness.Score':'Happiness Score',
                                  'Economy..GDP.per.Capita.':'Economy (GDP per Capita)',
                                  'Health..Life.Expectancy.':'Health (Life Expectancy)',
                                  'Trust..Government.Corruption.':'Trust (Government Corruption)',
                                  'Dystopia.Residual':'Dystopia Residual'
})

In [4]:
len(happiness_2015) + len(happiness_2016) + len(happiness_2017)

470

In [6]:
### Load economy data
disposable_income = pd.read_csv("../data/economy/household-disposable-income.csv")

### education data
adult_education = pd.read_csv("../data/education/adult-education-level.csv")

### Load environment data
air_pollution = pd.read_csv("../data/environment/air-pollution-exposure.csv")

### Load health data
alcohol_consumption = pd.read_csv("../data/body_health/alcohol-consumption.csv")
overweight_population = pd.read_csv("../data/body_health/overweight-population.csv")

### Load jobs data
avg_annual_hours = pd.read_csv("../data/jobs/average-annual-hours-worked-per-worker.csv")
income_inequality = pd.read_csv("../data/jobs/Income-inequality-Gini-coefficient.csv",encoding = 'unicode_escape')
unemployment = pd.read_csv("../data/jobs/Unemployment-total-of-labour-force.csv",encoding = 'unicode_escape')
unemployment = unemployment.drop(columns=['Unemployment, total (% of labour force)']).reset_index()
header = unemployment.iloc[0]
unemployment = unemployment[1:]
unemployment =unemployment.rename(columns = header)
 
### Society data
HDI = pd.read_csv("../data/society/Human-Development-Index-HDI.csv",encoding = 'unicode_escape')
HDI = HDI.drop(columns=['Human Development Index (HDI)']).reset_index()
header = HDI.iloc[0]
HDI = HDI[1:]
HDI = HDI.rename(columns = header)

In [7]:
overweight_population = overweight_population.rename(columns = { 
                                  "YEAR (CODE)":"TIME",
                                  'COUNTRY (CODE)': "LOCATION",
                                    'Numeric': "Value"}) 

In [8]:
#### Rename to unify column names
avg_annual_hours = avg_annual_hours.rename(columns = { 
                                  "COUNTRY":"LOCATION"}) 
income_inequality = income_inequality.rename(columns = { 
                                  "Country":"LOCATION"}) 

unemployment = unemployment.rename(columns = { 
                                  "Country":"LOCATION"}) 

HDI = HDI.rename(columns = {"Country":"LOCATION"}) 

## Data transformation
1. Filter support data records for only 2015,2016 and 2017
2. Filter based on relevant method of measure:
 - disposable_income was filtered to only contain rows from the category USD_CAP. 
 - adult_education was filtered to contain only rows from the category TRY. 
 - air_pollution, rows from the category PC_POP were kept 
 - avg_annual_hours, only rows from the TE category were kept. 


2. Group by Aberage, to simplier data: the country in the given year has only one value (using average of observation)
2. Outer Join support datasets based on the `Location` and `year`
3. Left Join (Return all rows from Main data) based on the `Location` and `year`


### Filter year data
We only need 2015,2016 and 2017 observations

In [9]:
"""
Filter the records in 2015, 2016 and 2017 only
Parameters:
---------------------------------------------
    years: a list
    data: the input dataset
    
Return:
---------------------------------------------
    the filer new dataset with records only in 2015, 2016 and 2017

"""


def filter_year(years, data):
    data = data[data.TIME.isin(years)]
    if len(data) == 0:
        print("useless data")
    return data

In [10]:
years = [2015, 2016, 2017]

disposable_income = filter_year(years, disposable_income)
adult_education = filter_year(years, adult_education)
air_pollution = filter_year(years, air_pollution)
alcohol_consumption = filter_year(years, alcohol_consumption)
overweight_population = filter_year(years, overweight_population)
avg_annual_hours = filter_year(years, avg_annual_hours)

income_inequality = income_inequality[['HDI Rank (2017)', 'LOCATION', '2015', '2016', '2017']]
unemployment = unemployment[['LOCATION','2015', '2016', '2017']]
HDI = HDI[['LOCATION','2015', '2016', '2017']]

### Filter Measures
- disposable_income was filtered to only contain rows from the category USD_CAP. 
- adult_education was filtered to contain only rows from the category TRY. 
- air_pollution, rows from the category PC_POP were kept
- avg_annual_hours, only rows from the TE category were kept. 

In [11]:
# 💕 new part!
disposable_income = disposable_income[disposable_income.MEASURE == 'USD_CAP']
adult_education = adult_education[adult_education.SUBJECT == 'TRY']
air_pollution = air_pollution[air_pollution.MEASURE == 'PC_POP']
avg_annual_hours = avg_annual_hours[avg_annual_hours.EMPSTAT == 'TE']

In [12]:
print(len(happiness_2015))
print(len(happiness_2016))
print(len(happiness_2017))

158
157
155


### Combine Happiness datasets

In [13]:
def input_values(year, data1, data2, country, column_name, row_count):
    try:
        data1.at[row_count, column_name] = data2.loc[data2['LOCATION'] == country][str(year)].tolist()[0]
    except IndexError: # cannot find value in dataset
        data1.at[row_count, column_name] = None
    return data1

def input_values2(year, data1, data2, country, column_name, row_count):
    try:
        data1.at[row_count, column_name] = data2.loc[data2['Country'] == country][column_name].tolist()[0]
    except IndexError: # cannot find value in dataset
        data1.at[row_count, column_name] = None
    return data1

def input_given_year(year, country, row_count, happiness,combined_df, income_inequality, HDI, unemployment):
    combined_df.at[row_count, 'LOCATION'] = country
    combined_df.at[row_count, 'TIME'] = year
    
    if year != 2017:
        combined_df.at[row_count, 'HDI Rank (2017)'] = None # we only have 2017 HDI rank
    else:
        try:
            combined_df.at[row_count, 'HDI Rank (2017)'] =  income_inequality.loc[income_inequality['LOCATION'] ==country]['HDI Rank (2017)'].tolist()[0]
        except IndexError: 
            combined_df.at[row_count, 'HDI Rank (2017)'] = None
    
    combined_df = input_values(year, combined_df, income_inequality, country, 'income_inequality', row_count)
    combined_df = input_values(year, combined_df, HDI, country, 'HDI_value', row_count)
    combined_df = input_values(year, combined_df, unemployment, country, 'unemployment', row_count)
    
    for column in ['Happiness Rank','Happiness Score','Economy (GDP per Capita)', 'Family',
                                   'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)',
                                   'Generosity', 'Dystopia Residual']:
        combined_df= input_values2(year, combined_df, happiness, country, column, row_count)
    
    return combined_df

In [14]:
combined_df = pd.DataFrame(columns=['LOCATION', 'TIME', 'Happiness Rank', 'HDI Rank (2017)','income_inequality', 
                                   'unemployment','HDI_value','Region','Happiness Score',
                                   'Economy (GDP per Capita)', 'Family',
                                   'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)',
                                   'Generosity', 'Dystopia Residual'])

row_count = -1
for index, row in happiness_2015.iterrows():
    
    # search country
    country = row['Country']
    region = row['Region']

    ## Fill 2015 observation
    row_count += 1
    combined_df = input_given_year(2015, country, row_count, happiness_2015, combined_df, 
                                   income_inequality, HDI, unemployment)
    combined_df.at[row_count, 'Region'] = region
    
    ## Fill 2016 observation
    row_count += 1
    combined_df = input_given_year(2016, country, row_count, happiness_2016, combined_df,
                                   income_inequality, HDI, unemployment)
    combined_df.at[row_count, 'Region'] = region
    
    ## Fill 2017 observation
    row_count += 1
    combined_df = input_given_year(2017, country, row_count, happiness_2017, combined_df,
                                  income_inequality, HDI, unemployment)
    combined_df.at[row_count, 'Region'] = region
    
    

In [15]:
combined_df.head(3)

,LOCATION,TIME,Happiness Rank,HDI Rank (2017),income_inequality,unemployment,HDI_value,Region,Happiness Score,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Switzerland,2015,1,None,8.4,4.8,0.942,Western Europe,7.587,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738
1,Switzerland,2016,2,None,7.5,4.9,0.943,Western Europe,7.509,1.52733,1.14524,0.86303,0.58557,0.41203,0.28083,2.69463
2,Switzerland,2017,4,2,7.5,4.8,0.944,Western Europe,7.494,1.56498,1.51691,0.858131,0.620071,0.367007,0.290549,2.27672


### Group by Average 
To simplier data: the country in the given year has only one value (using average of observation)

In [16]:
disposable_income = disposable_income.groupby(['LOCATION', 'TIME'])["Value"].mean()
adult_education = adult_education.groupby(['LOCATION', 'TIME'])["Value"].mean()
air_pollution = air_pollution.groupby(['LOCATION', 'TIME'])["Value"].mean()
alcohol_consumption = alcohol_consumption.groupby(['LOCATION', 'TIME'])["Value"].mean()
overweight_population = overweight_population.groupby(['LOCATION', 'TIME'])["Value"].mean()
avg_annual_hours = avg_annual_hours.groupby(['LOCATION', 'TIME'])["Value"].mean()

### Outer Join support datasets

In [17]:
new_df = pd.merge(disposable_income, adult_education,  
                  how='outer', 
                  left_on=['LOCATION','TIME'], 
                  right_on = ['LOCATION','TIME'],
                  suffixes=('_disposable_income', '_adult_education'))

new_df2 = pd.merge(alcohol_consumption, air_pollution,  
                  how='outer', 
                  left_on=['LOCATION','TIME'], 
                  right_on = ['LOCATION','TIME'],
                  suffixes=('_alcohol_consumption', '_air_pollution'))


In [18]:
new_df3 = pd.merge(overweight_population, avg_annual_hours,  
                  how='outer', 
                  left_on=['LOCATION','TIME'], 
                  right_on = ['LOCATION','TIME'],
                  suffixes=('_overweight_population', '_avg_annual_hours'))


In [19]:
final_data = pd.merge(new_df, new_df2,
                      left_on=['LOCATION','TIME'], 
                      right_on = ['LOCATION','TIME'],
                      how='outer')
final_data = pd.merge(final_data, new_df3,
                      left_on=['LOCATION','TIME'], 
                      right_on = ['LOCATION','TIME'],
                      how='outer')

In [20]:
final_data.reset_index(inplace=True)  

In [21]:
len(final_data['LOCATION'].unique()) # number of countries

207

In [22]:
final_data.head(10)

,LOCATION,TIME,Value_disposable_income,Value_adult_education,Value_alcohol_consumption,Value_air_pollution,Value_overweight_population,Value_avg_annual_hours
0,AUS,2015,37740.175079,42.888756,9.7,27.35672,64.033333,1683.55
1,AUS,2016,38658.848064,43.743900,9.4,26.00392,64.500000,1673.40
2,AUS,2017,39928.817097,45.355675,NaN,25.97999,NaN,1674.90
3,AUT,2015,34429.798275,30.550726,11.6,93.02611,53.900000,1500.00
4,AUT,2016,35653.315625,31.383963,11.8,87.03744,54.300000,1512.00
5,AUT,2017,36166.190395,32.394390,NaN,87.03754,NaN,1511.00
6,BEL,2015,32180.143590,36.857422,10.4,96.32111,59.200000,1545.00
7,BEL,2016,32965.901327,37.503956,NaN,93.32899,59.500000,1545.00
8,BEL,2017,33946.477041,40.257053,NaN,93.53039,NaN,1545.00
9,CAN,2015,31845.932814,55.173660,8.0,0.94055,63.666667,1712.00


### Using country code to match datasets
dataset resource:
https://gis.stackexchange.com/questions/1047/seeking-full-list-of-iso-alpha-2-and-iso-alpha-3-country-codes

In [23]:
country_code = pd.read_csv("../data/country_codes/wikipedia-iso-country-codes.csv")

In [24]:
country_code = country_code[['English short name lower case','Alpha-3 code']]
#country_code.head()

In [25]:
## create a directory to collect all useful countries
country_codes = {}
for code in final_data['LOCATION'].unique():
    try:
        country_name = country_code.loc[country_code['Alpha-3 code'] == code]['English short name lower case'].tolist()[0]
        if country_name == 'Congo, the Democratic Republic of the':
            country_name = 'Congo (Kinshasa)'
        elif country_name == 'Congo':
            country_name = 'Congo (Brazzaville)'
        
        country_name = country_name.replace(', Republic of','').replace(' the former Yugoslav Republic of','')
        country_name = country_name.replace(', Plurinational State of','').replace(', Islamic Republic of','').replace(" Democratic People's Republic of",'')
        
        country_codes[code] = country_name
    except IndexError: # cannt find relevant match
        print(code)
        continue
    country_codes['OECD'] = 'OECD countries' # special case
    
len(country_codes)

EA
EU
OAVG
EU28
G20
G7M
OECD
OECDE
WLD
SDF
SSD


197

In [26]:
#combined_df['LOCATION'].unique()

In [27]:
for index, row in final_data.iterrows():
    if row['LOCATION'] in country_codes:
        final_data.at[index,'LOCATION'] = country_codes[row['LOCATION']]
    else:
        print("Unable to match {}".format(row['LOCATION']))

Unable to match EA
Unable to match EA
Unable to match EA
Unable to match EU
Unable to match EU
Unable to match EU
Unable to match OAVG
Unable to match OAVG
Unable to match OAVG
Unable to match EU28
Unable to match EU28
Unable to match EU28
Unable to match G20
Unable to match G20
Unable to match G20
Unable to match G7M
Unable to match G7M
Unable to match G7M
Unable to match OECDE
Unable to match OECDE
Unable to match OECDE
Unable to match WLD
Unable to match WLD
Unable to match WLD
Unable to match SDF
Unable to match SDF
Unable to match SSD
Unable to match SSD


In [28]:
final_data = pd.merge(final_data, combined_df,
                      left_on=['LOCATION','TIME'], 
                      right_on = ['LOCATION','TIME'],
                      how='inner')  # right outer join

In [29]:
len(final_data['LOCATION'].unique())  # we finally match 140 countries

143

In [47]:
final_data.head(2)

,LOCATION,TIME,Value_disposable_income,Value_adult_education,Value_alcohol_consumption,Value_air_pollution,Value_overweight_population,Value_avg_annual_hours,Happiness Rank,HDI Rank (2017),income_inequality,unemployment,HDI_value,Region,Happiness Score,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Australia,2015,37740.175079,42.888756,9.7,27.35672,64.033333,1683.55,10,None,8,6.1,0.936,Australia and New Zealand,7.284,1.33358,1.30923,0.93156,0.65124,0.35637,0.43562,2.26646
1,Australia,2016,38658.848064,43.743900,9.4,26.00392,64.500000,1673.40,9,None,8.3,5.7,0.938,Australia and New Zealand,7.313,1.44443,1.10476,0.8512,0.56837,0.32331,0.47407,2.5465


In [31]:
df = final_data
df.shape

(426, 22)

## Data Cleaning
1. Detect incorrect values
2. Convert string type to numerical columns¶
3. Check duplicate rows
4. Check missing value
5. Handle missing value 
   - Drop rows 
   - Fill missing value


### Detect incorrect values
- Since we have lots of string, such as `..` in numiercal columns
- Replace them with `None` first, and will fill in them with other missing values in later steps

In [32]:
df = df.replace('..', None)

### Convert string type to numerical columns

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 426 entries, 0 to 425
Data columns (total 22 columns):
LOCATION                         426 non-null object
TIME                             426 non-null object
Value_disposable_income          97 non-null float64
Value_adult_education            120 non-null float64
Value_alcohol_consumption        104 non-null float64
Value_air_pollution              417 non-null float64
Value_overweight_population      282 non-null float64
Value_avg_annual_hours           106 non-null float64
Happiness Rank                   417 non-null object
HDI Rank (2017)                  125 non-null object
income_inequality                380 non-null object
unemployment                     402 non-null object
HDI_value                        402 non-null object
Region                           426 non-null object
Happiness Score                  417 non-null object
Economy (GDP per Capita)         417 non-null object
Family                           417 non-n

In [34]:
# the columns that shoube be in numerical type
for column in ['Happiness Rank', 'HDI Rank (2017)', 'income_inequality',
       'unemployment', 'HDI_value', 'Happiness Score',
       'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)',
       'Freedom', 'Trust (Government Corruption)', 'Generosity',
       'Dystopia Residual']:
    
    df[column] = df[column].astype(float)

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 426 entries, 0 to 425
Data columns (total 22 columns):
LOCATION                         426 non-null object
TIME                             426 non-null object
Value_disposable_income          97 non-null float64
Value_adult_education            120 non-null float64
Value_alcohol_consumption        104 non-null float64
Value_air_pollution              417 non-null float64
Value_overweight_population      282 non-null float64
Value_avg_annual_hours           106 non-null float64
Happiness Rank                   417 non-null float64
HDI Rank (2017)                  125 non-null float64
income_inequality                380 non-null float64
unemployment                     402 non-null float64
HDI_value                        402 non-null float64
Region                           426 non-null object
Happiness Score                  417 non-null float64
Economy (GDP per Capita)         417 non-null float64
Family                           41

### Check duplicate rows
we donot have duplicate rows

In [36]:
print(len(df)) # initial 
df = df.drop_duplicates()
print(len(df)) # after removing duplicates

426
426


### Check missing value

In [37]:
print("The number of missing values in each column: ")
# if sum >0, it means missing value
df.isnull().sum()

The number of missing values in each column: 


LOCATION                           0
TIME                               0
Value_disposable_income          329
Value_adult_education            306
Value_alcohol_consumption        322
Value_air_pollution                9
Value_overweight_population      144
Value_avg_annual_hours           320
Happiness Rank                     9
HDI Rank (2017)                  301
income_inequality                 46
unemployment                      24
HDI_value                         24
Region                             0
Happiness Score                    9
Economy (GDP per Capita)           9
Family                             9
Health (Life Expectancy)           9
Freedom                            9
Trust (Government Corruption)      9
Generosity                         9
Dystopia Residual                  9
dtype: int64

In [38]:
print("Print the missing ratio: ")
df.isnull().sum()/len(df)

Print the missing ratio: 


LOCATION                         0.000000
TIME                             0.000000
Value_disposable_income          0.772300
Value_adult_education            0.718310
Value_alcohol_consumption        0.755869
Value_air_pollution              0.021127
Value_overweight_population      0.338028
Value_avg_annual_hours           0.751174
Happiness Rank                   0.021127
HDI Rank (2017)                  0.706573
income_inequality                0.107981
unemployment                     0.056338
HDI_value                        0.056338
Region                           0.000000
Happiness Score                  0.021127
Economy (GDP per Capita)         0.021127
Family                           0.021127
Health (Life Expectancy)         0.021127
Freedom                          0.021127
Trust (Government Corruption)    0.021127
Generosity                       0.021127
Dystopia Residual                0.021127
dtype: float64

- We can see we have quite high missing ratio in `Value_disposable_income`, `Value_adult_education`, `Value_alcohol_consumption`, `Value_avg_annual_hours` and `Value_overweight_population`, and it is hard/unreasonable to fill in them.  
Thus, 

### Drop rows
To control the missing ratio

In [39]:
df = df[:126] #
df.shape

(126, 22)

In [40]:
print("Print the missing ratio: ")
df.isnull().sum()/len(df)

Print the missing ratio: 


LOCATION                         0.000000
TIME                             0.000000
Value_disposable_income          0.230159
Value_adult_education            0.047619
Value_alcohol_consumption        0.174603
Value_air_pollution              0.023810
Value_overweight_population      0.325397
Value_avg_annual_hours           0.158730
Happiness Rank                   0.000000
HDI Rank (2017)                  0.682540
income_inequality                0.031746
unemployment                     0.023810
HDI_value                        0.023810
Region                           0.000000
Happiness Score                  0.000000
Economy (GDP per Capita)         0.000000
Family                           0.000000
Health (Life Expectancy)         0.000000
Freedom                          0.000000
Trust (Government Corruption)    0.000000
Generosity                       0.000000
Dystopia Residual                0.000000
dtype: float64

In [41]:
len(df['LOCATION'].unique())

44

### Fill missing value 
Two stages:
- Stage 1: Given the country has at least one year observation, If one year is missing for a country, then average the other 2 years or 1 year. 
- Stage 2: After stage 1, if the country has no observation in all three years, then average the values from the countries which belong is the same region using the 'Region' attribute.
- Stage 3: If missing value for given region (i,e, both stage 1 and 2 fail), just use average value of that column

In [42]:
def isNaN(num):
    return num != num

def get_avg_region(data, column, country,region):
    region_data = data[data['Region'] == region]
    return region_data[column].mean()

"""
Stage 1 and Satge 2
"""
def fill_missing_value(df):
    final_df = df.copy()
    for index, row in final_df.iterrows():
        # the columns have missing value and need to fill
        for column in ['Value_disposable_income', 'Value_adult_education', 'Value_alcohol_consumption', 'Value_air_pollution',
                  'Value_avg_annual_hours','income_inequality', 'unemployment']:
            if row[column] == None or isNaN(row[column]):
                country = row['LOCATION']
                this_year = row['TIME']
                
                ## find ovservations for given country
                sub_data = final_df[final_df['LOCATION'] == country]
                #print(sub_data[['TIME', column]])
                ## look for the obs in other years
                sub_data = sub_data[sub_data['TIME'] != this_year]
                
                if not isNaN(sub_data[column].mean()):  
                    #print(sub_data[column].mean())
                    #print('fill using stage 1')
                    final_df.at[index, column] = sub_data[column].mean() # fill missing point based on stage 1 strategy
                
                else: # stage 2, since no observation for all three year
                    region = row['Region']
                    avg_region = get_avg_region(final_df, column, country, region)
                    #print('fill using stage 2')
                    final_df.at[index, column] = avg_region
                    if isNaN(avg_region):  # Stage 3: If still missing, we fill missing point
                        print('need fill using stage 3')
                        continue
                        
        
            #print('---------------------------------------------------------------')
        
    
    return final_df
    
    


In [43]:
# 💕 new part!
final_df = fill_missing_value(df)

need fill using stage 3
need fill using stage 3
need fill using stage 3
need fill using stage 3
need fill using stage 3
need fill using stage 3
need fill using stage 3
need fill using stage 3
need fill using stage 3
need fill using stage 3
need fill using stage 3
need fill using stage 3
need fill using stage 3
need fill using stage 3
need fill using stage 3


In [44]:
final_df.isnull().sum()/len(final_df)

LOCATION                         0.000000
TIME                             0.000000
Value_disposable_income          0.039683
Value_adult_education            0.015873
Value_alcohol_consumption        0.000000
Value_air_pollution              0.000000
Value_overweight_population      0.325397
Value_avg_annual_hours           0.063492
Happiness Rank                   0.000000
HDI Rank (2017)                  0.682540
income_inequality                0.000000
unemployment                     0.000000
HDI_value                        0.023810
Region                           0.000000
Happiness Score                  0.000000
Economy (GDP per Capita)         0.000000
Family                           0.000000
Health (Life Expectancy)         0.000000
Freedom                          0.000000
Trust (Government Corruption)    0.000000
Generosity                       0.000000
Dystopia Residual                0.000000
dtype: float64

In [45]:
"""
Stage 3:
"""
for column in ['Value_disposable_income', 'Value_adult_education', 'Value_alcohol_consumption', 'Value_air_pollution',
              'Value_avg_annual_hours','income_inequality', 'unemployment']:
    final_df[column].fillna((final_df[column].mean()), inplace=True)

### output the cleaned dataset!

In [46]:
final_df.to_csv('../data/final_data.csv', index = None)